NNDL Lab 9

In [2]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras import layers, regularizers, Model
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score

# ===== MOUNT GOOGLE DRIVE (if using Colab) =====
from google.colab import drive
drive.mount('/content/drive')

# ===== DATASET PATH =====
dataset_path = "/content/drive/MyDrive/NNDL Dataset/Dataset"

# ===== CATEGORIES =====
categories = ["gond painting", "kalighat painting", "kangra painting", "kerala mural",
              "madhubani painting", "mandana art drawing", "pichwai painting", "warli painting"]

# ===== IMAGE LOADING SETTINGS =====
img_size = (64, 64)  # Reduce image size
max_images_per_class = 200  # Limit per class

def load_images(folder_path, img_size, max_images):
    images, labels = [], []
    for category in categories:
        category_path = os.path.join(folder_path, category)
        if not os.path.exists(category_path):
            continue
        files = [f for f in os.listdir(category_path) if f.lower().endswith((".jpg", ".jpeg", ".png"))][:max_images]
        for filename in files:
            img_path = os.path.join(category_path, filename)
            img = load_img(img_path, target_size=img_size)
            img_array = img_to_array(img) / 255.0  # Normalize
            images.append(img_array)
            labels.append(categories.index(category))
    return np.array(images), np.array(labels)

# ===== LOAD DATASET =====
X, y = load_images(dataset_path, img_size, max_images_per_class)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

# ===== DEFINE CNN MODEL =====
def create_cnn(input_shape):
    inputs = layers.Input(shape=input_shape)
    x = layers.Conv2D(32, (3, 3), activation='relu')(inputs)
    x = layers.MaxPooling2D((2, 2))(x)
    x = layers.Conv2D(64, (3, 3), activation='relu')(x)
    x = layers.MaxPooling2D((2, 2))(x)
    x = layers.Flatten()(x)
    feature_output = layers.Dense(128, activation='relu', name='feature_layer')(x)
    x = layers.Dense(len(categories), activation='softmax')(feature_output)
    return Model(inputs=inputs, outputs=x)

# ===== OPTIMIZERS TO TEST =====
optimizers = {
    "Adam": tf.keras.optimizers.Adam(),
    "SGD": tf.keras.optimizers.SGD(),
    "RMSprop": tf.keras.optimizers.RMSprop()
}

# ===== CLASSIFIERS =====
classifiers = {
    "kNN": KNeighborsClassifier(n_neighbors=3),
    "Decision Tree": DecisionTreeClassifier(),
    "Random Forest": RandomForestClassifier(n_estimators=50),
    "SVM": SVC(kernel='linear'),
    "Naïve Bayes": GaussianNB()
}

# ===== TRAIN & EVALUATE CNN FOR EACH OPTIMIZER =====
results = {}
early_stopping = EarlyStopping(monitor='val_loss', patience=2, restore_best_weights=True)

for opt_name, optimizer in optimizers.items():
    print(f"\n🔹 Training with {opt_name} optimizer...")

    # Train CNN
    cnn_model = create_cnn((64, 64, 3))
    cnn_model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    cnn_model.fit(X_train, y_train, epochs=5, validation_data=(X_val, y_val), batch_size=16, callbacks=[early_stopping], verbose=1)

    # Extract features
    feature_extractor = Model(inputs=cnn_model.input, outputs=cnn_model.get_layer('feature_layer').output)
    X_train_features = feature_extractor.predict(X_train, batch_size=32)
    X_val_features = feature_extractor.predict(X_val, batch_size=32)

    # Train classifiers
    classifier_results = {}
    for clf_name, clf in classifiers.items():
        clf.fit(X_train_features, y_train)
        y_pred = clf.predict(X_val_features)
        accuracy = accuracy_score(y_val, y_pred)
        classifier_results[clf_name] = accuracy

    results[opt_name] = classifier_results

# ===== PRINT RESULTS =====
for opt, res in results.items():
    print(f"\n🔹 Results for {opt} Optimizer:")
    for clf_name, acc in res.items():
        print(f"   ▶️ {clf_name}: Accuracy = {acc:.4f}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


/usr/local/lib/python3.11/dist-packages/PIL/Image.py:1045: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(



🔹 Training with Adam optimizer...
Epoch 1/5
45/45 ━━━━━━━━━━━━━━━━━━━━ 7s 111ms/step - accuracy: 0.1951 - loss: 2.1660 - val_accuracy: 0.3258 - val_loss: 1.8834
Epoch 2/5
45/45 ━━━━━━━━━━━━━━━━━━━━ 4s 96ms/step - accuracy: 0.3886 - loss: 1.7594 - val_accuracy: 0.3989 - val_loss: 1.7337
Epoch 3/5
45/45 ━━━━━━━━━━━━━━━━━━━━ 7s 136ms/step - accuracy: 0.4559 - loss: 1.4909 - val_accuracy: 0.4101 - val_loss: 1.6000
Epoch 4/5
45/45 ━━━━━━━━━━━━━━━━━━━━ 4s 94ms/step - accuracy: 0.5451 - loss: 1.2930 - val_accuracy: 0.4551 - val_loss: 1.5208
Epoch 5/5
45/45 ━━━━━━━━━━━━━━━━━━━━ 4s 94ms/step - accuracy: 0.6911 - loss: 0.9467 - val_accuracy: 0.4831 - val_loss: 1.5557
23/23 ━━━━━━━━━━━━━━━━━━━━ 2s 74ms/step
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step

🔹 Training with SGD optimizer...
Epoch 1/5
45/45 ━━━━━━━━━━━━━━━━━━━━ 5s 98ms/step - accuracy: 0.1923 - loss: 2.0628 - val_accuracy: 0.2247 - val_loss: 2.0218
Epoch 2/5
45/45 ━━━━━━━━━━━━━━━━━━━━ 4s 89ms/step - accuracy: 0.2197 - loss: 1.9980 - val_accur